# MySQL 실습 프로젝트

In [1]:
!pip install pymysql > /dev/null
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)

In [3]:
import random
random.seed(2021)

## Products Table 생성

- 3가지 카테고리에 총 30가지 이상의 제품을 갖출 것
        products(pid, pname, pprice, pcategory, pcost)

In [4]:
sql = '''
    create table if not exists products (
        pid int primary key auto_increment,
        pname varchar(40) not null,
        pprice int unsigned not null,
        pcategory varchar(20) not null,
        pcost int unsigned not null
    );
'''

In [6]:
cur = conn.cursor()
cur.execute(sql)

0

In [7]:
kitchen_goods = ['냉장고','김치냉장고','식기세척기','전자레인지','오븐',
                 '전기레인지','정수기','얼음정수기','에어프라이어','압력밥솥']
living_goods = ['세탁기','의류건조기','스타일러','로봇청소기','워시타워',
                '안마의자','가습기','스팀청소기','공기청정기','에어컨']
health_goods = ['훌라후프','줄넘기','스텝퍼','러닝머신','짐볼',
                '덤벨','로잉머신','스쿼트머신','싯업벤치','사이클']
category_names = ['주방가전','생활가전','운동기구']

In [8]:
for item in kitchen_goods:
    price = random.randint(100,500) * 1000
    rate = random.randint(85,95)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '주방가전', cost)

냉장고 306000 주방가전 290700
김치냉장고 378000 주방가전 336420
식기세척기 226000 주방가전 214700
전자레인지 117000 주방가전 107640
오븐 342000 주방가전 321480
전기레인지 132000 주방가전 118800
정수기 238000 주방가전 211820
얼음정수기 343000 주방가전 294980
에어프라이어 182000 주방가전 158340
압력밥솥 127000 주방가전 109220


In [9]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([kitchen_goods, living_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,500) * 1000
        rate = random.randint(85, 95)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

## Sales Table 생성

- 5개 이상의 회사와 거래할 것
- 월 10건 이상의 매출 건수를 가져야함(2020.01~2020.12)
        sales(sid, sdate, scompany, sid, sunit)

In [10]:
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [12]:
cur.execute(sql)
conn.commit()

In [13]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['신한은행','국민은행','우리은행','기업은행','하나은행']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-13 우리은행 24 6
2020-1-21 기업은행 15 20
2020-1-27 기업은행 40 10
2020-1-4 우리은행 11 15
2020-1-30 하나은행 20 12
2020-1-21 신한은행 13 17
2020-1-31 신한은행 24 11
2020-1-19 신한은행 34 4
2020-1-6 기업은행 16 12
2020-1-7 국민은행 13 4


In [14]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(1,30)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

## Sales_book View 생성
- Join Products Table & Sales Table

In [15]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [16]:
cur = conn.cursor()
cur.execute(sql)

0

In [17]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [18]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,101,2020-01-01,우리은행,24,러닝머신,운동기구,455000,11,404950,5005000,550550
1,106,2020-01-02,하나은행,13,스타일러,생활가전,340000,17,292400,5780000,809200
2,105,2020-01-02,신한은행,14,로봇청소기,생활가전,431000,11,400830,4741000,331870
3,108,2020-01-08,우리은행,20,에어컨,생활가전,199000,17,173130,3383000,439790
4,109,2020-01-09,우리은행,6,전기레인지,주방가전,425000,14,391000,5950000,476000


In [19]:
del df[3]
del df[0]
df.head()

,1,2,4,5,6,7,8,9,10
0,2020-01-01,우리은행,러닝머신,운동기구,455000,11,404950,5005000,550550
1,2020-01-02,하나은행,스타일러,생활가전,340000,17,292400,5780000,809200
2,2020-01-02,신한은행,로봇청소기,생활가전,431000,11,400830,4741000,331870
3,2020-01-08,우리은행,에어컨,생활가전,199000,17,173130,3383000,439790
4,2020-01-09,우리은행,전기레인지,주방가전,425000,14,391000,5950000,476000


In [20]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,우리은행,러닝머신,운동기구,455000,11,404950,5005000,550550
1,2020-01-02,하나은행,스타일러,생활가전,340000,17,292400,5780000,809200
2,2020-01-02,신한은행,로봇청소기,생활가전,431000,11,400830,4741000,331870
3,2020-01-08,우리은행,에어컨,생활가전,199000,17,173130,3383000,439790
4,2020-01-09,우리은행,전기레인지,주방가전,425000,14,391000,5950000,476000


In [21]:
df.to_csv('sales_book.csv', index=False)

## 마무리

In [22]:
cur.close()
conn.close()